# PROJECT : IMAGE REGENERATION 

This project contains scraping and mining of image data from a social network site, in this case pinterest, processing and training the images, and finally generating new images from the already created image dataset.

It will require the use of a list of libraries such as pandas and numpy etc, a scraper - BeautifulSoup API & Selenium, Selenium chrome web driver and implement a neural network model of GANs.

    > pip install selenium OR conda install -c conda-forge selenium
    
The probable sites were Reddit/Pinterest - limited access & scraping took a lot of RAM, Unsplash - free access and greatly pixelated images. Unsplash was the site of choice

Inspiration/Refernce : 

    > https://www.nypl.org/blog/2014/06/24/peeling-painted-layers-nyc-google-street-view
    > https://gizmodo.com/this-nifty-tool-turns-google-street-view-into-a-graffit-1595919773
    

In [3]:
#libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [10]:
import time
import datetime

In [28]:
import requests
import os
import io
from PIL import Image
import hashlib

In [34]:
#initiate the web driver
D_PATH = 'chrome_driver/chromedriver.exe'
service = Service(D_PATH)
service.start()

#confirm driver is running
wd = webdriver.Remote(service.service_url)
wd.quit()


### SCRAPING UNSPLASH.COM

In [50]:
#function below scrolls through the page selected page
#url : https://unsplash.com/s/photos/graffiti-wall?orientation=landscape
#use css/class name selector as the page is not static

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3): 
    #scrolls to end of page
    def bottomscroll_func(wd, scroll_point):
        wd.execute_script(f'window.scrollTo(0, {scroll_point});')
        time.sleep(sleep_between_interactions)
        
    #site url
    #search_url = f'https://unsplash.com/s/photos/graffiti-wall?orientation=landscape'
    search_url = f'https://unsplash.com/s/photos/{query}'
    #loading the sitepage
    wd.get(search_url)
    time.sleep(sleep_between_interactions)

    image_urls = set()
    image_count = 0
    number_results = 0
        
    for i in range(0,20):
        bottomscroll_func(wd, i*1000)
        time.sleep(5)
        #locating elements by class name/ css selector - check target site image format
        image_thumb = wd.find_elements_by_css_selector('img._2UpQX')
        #image_thumb = wd.find_element_by_css_selector('img._2UpQX')
        #image_thumb = wd.find_elements_by_class_name('_2UpQX')
        #image_thumb = wd.find_element_by_class_name('_2UpQX')
        #session break during search
        time.sleep(5)
            
        for img in image_thumb:
            print(img)
            print(img.get_attribute('src'))
            image_urls.add(img.get_attribute('src'))
            image_count = len(image_urls)
            number_results = image_count
            time.sleep(.5)
        print(f'Found: {number_results} search results.\n ...Extracting Links..')
    return image_urls

In [37]:
#source : 
def persist_image(folder_path:str, url:str):
    try:
        #the headers below can be swapped with the desired browser
        #current browser version
        headers = {'User-agent' : 'Chrome/89.0.4389.82'}
        image_content = requests = requests.get(url, headers=headers).content
    except Exception as e:
        print(f'ERROR  - {e} \n - Unable to download {url}')
        
    try:
        #load images in byte form
        image_file = io.BytesIO(image_content)
        #convert images to rgb format
        image = Image.open(image_file).convert('RGB')
        #encrypt and save the images in jpeg format
        file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, 'JPEG', quality=85)
        print(f'COMPLETE - saved {url} - as {file_path}')
    except Exception as e:
        print(f'ERROR - {e} \n- Unable to save {url}')
        

In [54]:
def image_search_and_download(search_term:str, driver_path:str, target_path='downloaded_images/', number_images=5): #200
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))
    
    #check if folder exists
    if not os.path.exists(target_folder):
        #creates new folder
        os.makedirs(target_folder)
    
    #access to driver directory and executes driver
    with webdriver.Chrome(executable_path=driver_path) as wd:
        result = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=3)
        
        #download saved images
        for element in result:
            persist_image(target_folder, element)

In [ ]:
#terms for searching unsplash using query
search_terms = ['graffiti wall','graffiti','graffiti art','mural','street art'] #graffiti-wall?orientation=landscape

for search_term in search_terms:
    image_search_and_download(search_term=search_term, driver_path=D_PATH)

<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="609d9048-f592-40fd-b810-7c59ec31ce1b")>
https://images.unsplash.com/photo-1465859674575-26d1bbca2bc1?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MXx8Z3JhZmZpdGklMjB3YWxsfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="f8f54071-b758-4de9-8b1d-2be714e3b029")>
https://images.unsplash.com/photo-1609718467864-21e4a8a4ec71?ixid=MXwxMjA3fDB8MHxzZWFyY2h8M3x8Z3JhZmZpdGklMjB3YWxsfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="fdcf5947-2c5f-46b5-83f6-bad4d062b1b7")>
https://images.unsplash.com/photo-1609718467369-0e2b3d8cb0e4?ixid=MXwxMjA3fDB8MHxzZWFyY2h8NXx8Z3JhZmZpdGklMjB3YWxsfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="6a7d98

<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="f8f54071-b758-4de9-8b1d-2be714e3b029")>
https://images.unsplash.com/photo-1609718467864-21e4a8a4ec71?ixid=MXwxMjA3fDB8MHxzZWFyY2h8M3x8Z3JhZmZpdGklMjB3YWxsfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="fdcf5947-2c5f-46b5-83f6-bad4d062b1b7")>
https://images.unsplash.com/photo-1609718467369-0e2b3d8cb0e4?ixid=MXwxMjA3fDB8MHxzZWFyY2h8NXx8Z3JhZmZpdGklMjB3YWxsfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="6a7d981d-2d4f-45bc-bcc3-63150e6e0545")>
https://images.unsplash.com/photo-1609718467345-633f9af2997f?ixid=MXwxMjA3fDB8MHxzZWFyY2h8N3x8Z3JhZmZpdGklMjB3YWxsfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="2daab0

<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="553b0e1f-101f-43b8-9330-7aff0be04aca")>
https://images.unsplash.com/photo-1549890259-292c6c7fa0ef?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTl8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="8a92a7f0-cfd7-410e-9997-d8c03a336198")>
https://images.unsplash.com/photo-1609712929784-88d98c86f34c?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjF8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="fe233443-aee7-4008-affe-c1a8256a5aca")>
https://images.unsplash.com/photo-1614210400777-70f9e361e46e?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjR8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334d

<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="c7df5bb4-3fb5-4034-ad24-ea6aaf74dfcd")>
https://images.unsplash.com/photo-1553503995-c6e407a983b9?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTB8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="6fd849e4-632d-4fbd-918f-d3072a751680")>
https://images.unsplash.com/photo-1529990109904-a05c9b7ee458?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTJ8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="f3c58a06-aed0-49d4-bfd2-f2e26f88d4b1")>
https://images.unsplash.com/photo-1612676280741-02a44586fbde?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTR8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334d

<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="836e563e-8ffb-42f7-817a-caff57237dab")>
https://images.unsplash.com/photo-1614210358042-933d978732be?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTZ8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="f56e7728-33b8-4a2a-b393-955bd1517627")>
https://images.unsplash.com/photo-1501470364612-5a3e19052dc1?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTh8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b334df38", element="497cdeb4-123c-4646-a0df-2f8c54de5378")>
https://images.unsplash.com/photo-1502869051556-11e86743bbbb?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjB8fGdyYWZmaXRpJTIwd2FsbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="e1fe260511f3e1429b2a8132b3

...

<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="c8fde106-018a-49bd-8972-db3ee84d5e83")>
https://images.unsplash.com/photo-1495482432709-15807c8b3e2b?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MXx8Z3JhZmZpdGl8ZW58MHx8MHw%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="b3d4b3af-db2c-402c-b03c-8371bc35f3f8")>
https://images.unsplash.com/photo-1515584292151-a8ce75f68eff?ixid=MXwxMjA3fDB8MHxzZWFyY2h8NHx8Z3JhZmZpdGl8ZW58MHx8MHw%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="fe5a1f4b-0f6c-4db5-87be-d895eefbf90d")>
https://images.unsplash.com/photo-1565279427445-10c13a1d1f82?ixid=MXwxMjA3fDB8MHxzZWFyY2h8Nnx8Z3JhZmZpdGl8ZW58MHx8MHw%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="96a1dfa8-ce24-4157-9a07-

<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="7a4c081e-cdfc-4c8d-a7d0-f2834a41883a")>
https://images.unsplash.com/photo-1487452066049-a710f7296400?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTN8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="7f90a9ba-37f0-490f-afcd-bef1192c90d8")>
https://images.unsplash.com/photo-1543097840-fc84594dd9e6?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTR8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="712f4b52-6119-48bd-8ed5-6f7e95dd9c1c")>
https://images.unsplash.com/photo-1516810714657-e654b97f1d80?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTZ8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="f345fd88-180f-4fe1-a88e-0afe8b214

<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="2d8fbac2-bc3f-4134-82a1-bbe7c515dd74")>
https://images.unsplash.com/photo-1534162169564-158041fc3992?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MTl8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
Found: 20 search results.
 ...Extracting Links..
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="c8fde106-018a-49bd-8972-db3ee84d5e83")>
https://images.unsplash.com/photo-1495482432709-15807c8b3e2b?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MXx8Z3JhZmZpdGl8ZW58MHx8MHw%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="725f5a97-3fb4-4420-8418-d4cd437c54b8")>
https://images.unsplash.com/photo-1530406831759-15c5c0cbce8b?ixid=MXwxMjA3fDB8MHxzZWFyY2h8OXx8Z3JhZmZpdGl8ZW58MHx8MHw%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa

<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="3e6e6051-db3d-4bd5-b55e-6a0c07dd54d2")>
https://images.unsplash.com/photo-1604716053460-3f66248bf8de?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjZ8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="9068c5f1-0da2-4c99-8617-cb8511cd8501")>
https://images.unsplash.com/photo-1603328203114-8b9c370b374d?ixid=MXwxMjA3fDB8MHxzZWFyY2h8Mjh8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="f345fd88-180f-4fe1-a88e-0afe8b214967")>
https://images.unsplash.com/photo-1540678931042-41a977082abf?ixid=MXwxMjA3fDB8MHxzZWFyY2h8Mnx8Z3JhZmZpdGl8ZW58MHx8MHw%3D&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="c9584254-b9b7-41ea-8d07-6dbe

<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="a011df05-4a44-4d23-8a54-0dfd02ee908f")>
https://images.unsplash.com/photo-1604715892639-52a6cafba47e?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjF8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="25beec84-7fea-4f20-bab4-e21ab4cab92d")>
https://images.unsplash.com/photo-1488409688217-e6053b1e8f42?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjN8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="d1649ee1-02fd-43e2-8f0d-64f82846e4b8")>
https://images.unsplash.com/photo-1490535004195-099bc723fa1f?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MjV8fGdyYWZmaXRpfGVufDB8fDB8&ixlib=rb-1.2.1&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="6fe3f92c7842de48ec4aa2c2043689f9", element="4ad7689a-5a32-4d28-8e89-271ade

##### comments: 

multiple searches for results. output : 

    > the site loads and reloads with 5 sec breaks
    > loads 100 images
    
